In [4]:
import MarketDataFetcher
import asyncio
import nest_asyncio
import utils
import json
import numpy as np
import pickle

nest_asyncio.apply()

obj = MarketDataFetcher.FuturesMarket()
data = asyncio.run(obj.fetch_klines_limit(symbol='BTCUSDT', interval='3m', limit=1_000))
array = np.array(data, float)

In [ ]:
data = asyncio.run(obj.fetch_24hr_ticker())
symbols = []
for i in data:
    symbol = i['symbol']
    if symbol[-4:] == 'USDT':
        symbols.append(symbol)
print(symbols)

In [ ]:
asyncio.run(utils._wait_until_next_interval(time_unit='minute', interval=2))

In [ ]:
import datetime
getattr(datetime.datetime, 'hour', None)

In [ ]:
import utils

utils._info_kline_columns()

In [1]:
import run_test
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [ ]:
asyncio.run(run_test.SymbolConfig.validate_config())

In [ ]:
ins_market = MarketDataFetcher.FuturesMarket()
asyncio.run(ins_market.fetch_24hr_ticker())

In [1]:
import run_test
import nest_asyncio
import asyncio

nest_asyncio.apply()

In [ ]:
asyncio.run(run_test.SymbolConfig.validate_config())

In [2]:
import run_test
import nest_asyncio
import asyncio

nest_asyncio.apply()
run_test.OrderConfig.validate_config()

Error : OrderConfig.margin_mode
 1. 오류내용: 입력오류
 2. 해결방법: cross, isolated" 중 1개 선택 입력
 3. 현재설정: a


ValueError: OrderConfig 검증 실패

In [ ]:
a = None
a>0

In [2]:
mode_allowed_range = ['cross', 'isolated']
a = 'cross'

In [3]:
if a in mode_allowed_range:
    print(True)

True
